# Importing Libraries


In [20]:
import numpy as np 
import pandas as pd
from sklearn.tree import DecisionTreeClassifier

# Downloading Data

For this exercise, we will be using the Poe dataset. This dataset is composed of the word frequency of texts written by different authors and one text that is written by an unknown author. It is beleived to be the case that this work was written by Poe, but the purpose of this exercise is to either prove or disprove that hypothesis.

In [21]:
import csv
import re

# name of file
filename = "Poe_NotPoe_50words.csv"

# read in csv
pdf = pd.read_csv(filename, index_col=0)

# for our dataset, we have two rows "Total" and "Average", which should be removed before analysis
pdf = pdf.drop(["Total","Average"])

# create X as a numpy array
X = pdf.to_numpy()
y = []
for row in pdf.index:
    y.append(re.sub("_.+","",row))

y = pd.DataFrame(y)
y = y.to_numpy()

# Set up Decision Tree

We now need to create the splits for the decision tree. A decision tree is similar to a Search Tree or a flowchart, where we will be analysing certain feature values in order to conclude what class a certain sample belongs to. For instance, if we created a decision tree for figuring out the best course of treatment for COVID-19, we could have features of patient age, sex, and length of infection in order to determine what the best course of treatment is: treatment A or treatment B.

Something we will come across later, but is helpful to mention now, is the idea of entropy. Entropy is the value of uncertatinty at a certain node within a decision tree. An entropy of 0 shows that all samples at a node are of the same class, where if the entropy is higher than 0, there are samples of multiple classes at the same node. Because of this, leaf nodes almost always (if not always) have an entropy equal to 0, while branch nodes will have an entropy greater than 0. The higher the entropy, the more variation there is in classes at the node.

## Test Train Split

In order to train our model, we need to split our data into a test set and a train set. For this, we will use the train_test_split function from sklearn, and create 4 variables:
X_trainset: The features of the training set
X_testset: The features of the testing set
y_trainset: The class values of the training set
y_testset: The class values of the testing set

We will be passing in the arguments `test_size=1` to signify that we want 1 instance of our data to be used for testing and thus the rest of our data will be used to train our model, and the argument `random_state=0` gives a seed for creating splits that we can use repeatedly to recreate the same splits. We are then forcing our UNKNOWN data point to be the test set in the lines below, overriding the X_testset and y_testset that were just created.

In [22]:
from sklearn.model_selection import train_test_split


X_trainset, X_testset, y_trainset, y_testset = train_test_split(X[0:85], y[0:85], test_size=1, random_state=0)

# force UNKNOWN to be in testset
X_testset = X[85:86]
y_testset = y[85:86]


#print(X_trainset)
#print(X_testset.shape)
#print(y_trainset.shape)
#print(X_testset)

#print('Shape of X training set {}'.format(X_trainset.shape),'&',' Size of Y training set {}'.format(y_trainset.shape))
#print('Shape of X testing set {}'.format(X_testset.shape),'&',' Size of Y testing set {}'.format(y_testset.shape))


In [23]:
poeTree = DecisionTreeClassifier(criterion="entropy", max_depth = 20)
poeTree # it shows the default parameters

poeTree.fit(X_trainset,y_trainset)


DecisionTreeClassifier(criterion='entropy', max_depth=20)

# Prediction Time

In [24]:
predTree = poeTree.predict(X_testset)

# print (predTree [10:15])
# print (y_testset [10:15])
print (predTree)
print("y_testset")
print (y_testset)

['NotPoe']
y_testset
[['UNKNOWN']]


# Evaluation

When evaluating a decision tree, one metric to use to determine its accuracy is its accuracy! In this case, the accuracy does not accurately reflect the true value of our test set (as we only have a prediction of our test set instance) but in most other situations, accuracy is an incredibly important metric. You of course want this number to be as close to 100% as possible in order to have the most accurate model possible. It of course is easy to accomplish this by over-fitting our model to our data, so I encourage you to look up other metrics and analyze this model and your own with **F1 Score**, **Mean Squared Error**, or by creating a **Confusion Matrix**.

In [25]:
from sklearn import metrics
import matplotlib.pyplot as plt
print("DecisionTrees's Accuracy: ", metrics.accuracy_score(y_testset, predTree))

DecisionTrees's Accuracy:  0.0


# Visualization

Decision Trees are super helpful in the pursuit of organizing data into classes and determining the driving factors of such clusters. Decision Trees work similarly to flow charts, where we start at the top (root) node and work our way down with boolean (True/False) arguments. So, without further ado, let's start plotting.

In [26]:
# Notice: You might need to uncomment and install the pydotplus and graphviz libraries if you have not installed these before
#!conda install -c conda-forge pydotplus -y
#!conda install -c conda-forge python-graphviz -y

from  io import StringIO
import pydotplus
import matplotlib.image as mpimg
from sklearn import tree
%matplotlib inline 

In [27]:
dot_data = StringIO()
filename = "poeMelvilleTree.png"
featureNames = pdf.columns[0:40]
#out=tree.export_graphviz(poeTree,feature_names=featureNames, out_file=dot_data, class_names= np.unique(y_trainset), filled=True,  special_characters=True,rotate=False)  
out=tree.export_graphviz(poeTree,feature_names=featureNames, out_file=dot_data, class_names= np.unique(y_trainset), filled=True,  special_characters=True,rotate=False)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_png(filename)
img = mpimg.imread(filename)
plt.figure(figsize=(200, 400))
plt.imshow(img,interpolation='nearest')

# Results

As mentioned earlier, Decision Trees are incredibly powerful tools in the realm of data analysis, but it may take a second to get used to deciphering these visualizations. Let's take a look at our example above. At our root node, you can see that we start out with an entropy of 0.677, which is definitely not 0. You can see as we progress down the tree, sometimes this value can increase or decrease, but should approach 0 as we approach each leaf node. You can see that we start with 84 samples at our root node, which is becuase of our forcing all but 2 of our samples to be in the training set. Of these 84 instances, 68 are NotPoe and 16 are Poe. We start with an entropy of 0.702 and the class shows what class the majority of our samples are in. The top "upon <= 0.018" shows the starting argument of our decision tree. Given this inequality, if it is true for a sample, we follow the left arrow and if it is false, we follow the right arrow. After creating this Decision Tree, we can take any new instance - UNKNOWN - and run it through this tree in order to see why the model previously predicted it as NotPoe. Starting at the root node, we look at the value of the "upon" feature for UNKNOWN which equals .0033. Since .0033 < .018, this node's inequality is true and thus we follow the decision tree to the left. Looking at this node, we now have an entropy of .316 (better!) with 70 samples. If we had to cut the tree here for some reason (inability to test features, less than optimal number of features, etc), and had a sample that led to this node, we would have a 66/70 = 94% chance that the sample is NotPoe and 4/70 = 6% chance that the sample is Poe. However, we do have another inequality so rather than guessing here, let's continue down the tree. The inequality reads "at <= 0.032", and looking at UNKNOWN, we see that "at" = 0.02, which makes the inequality true. In this case, we traverse the left arrow and find ourselves at a new node with yet again entropy > 0. It is worth noting that at this node, the sample has a 64/65 = 98% chance to be NotPoe, so if the process of determining the feature's value and traversing the tree was time sensitive, depending on the use-case, one could deduce that this sample is most likely NotPoe. This node's inequality reads "some <= 0.003" and looking at UNKNOWN, "some" = 0.0067, which makes the inequality false, moving to the right, we see that this node is a leaf node with entropy 0, so the model guarantees that UNKNOWN is not written by Poe.